## Parse json specification file without any package

In [1]:
# To iterate files in input directory
import sys
import os
import numpy as np
import pandas as pd


In [2]:
# Take fixed width as command line argument from user
f_width=5 #int(sys.argv[1])

# Define input and output directories. 
ipDir = '/Users/harmeenkaur/Documents/input-test/'
opDir =  '/Users/harmeenkaur/Documents/output-test/'

# Read the specification file
with open('./spec.json') as f:
    lines = f.read()

# Remove newlines
lines = lines.replace("\n", "")

In [3]:
# Extract column names and widths
# Considering the fact that they are contained in lists with the first being ColumnNames and second being Offsets
cols = lines.split('[')[1].split(']')[0].replace(" ", "")
offsets = lines.split('[')[2].split(']')[0].replace(" ", "")

# OR this can be extracted as below. This is a bit more flexible since ordering is not important for column names and offsets.

# Split based on "ColumnNames", extract the 1st value, split is again on ] since the values are contained in a list. Split again on the [ and extract the
# first value. Remove extra spaces and quotes
cols=lines.split("ColumnNames",1)[1].split(']')[0].split('[')[1].replace(" ", "").replace("\"", "")

# Convert cols into a list
headers=list(cols.split(','))

# Split based on Offsets and extract the first value. Then split on closing ] to extract the entire block, split on opening [ and get the first value.
# Remove extra spaces and quotes
offsets = lines.split("Offsets",1)[1].split(']')[0].split('[')[1].replace(" ", "").replace("\"", "")
#print(offsets)

# Split based on FixedWidthEncoding and extract the first value. Then split on , to get the block and finally on :
f_width_enc = lines.split("FixedWidthEncoding",1)[1].split(',')[0].split(':')[1].lstrip()
#print(f_width_enc+'\n')

# Extract IncludeHeader in a similar way as above
inc_header = lines.split("IncludeHeader",1)[1].split(',')[0].split(':')[1].lstrip()
#print(inc_header+'\n')

# Extract DelimitedEncoding in a similar way
dem_enc = lines.split("DelimitedEncoding",1)[1].split('}')[0].replace("\":", "").lstrip()
#print(dem_enc+'\n')

# Same logic as above
# Convert offsets into list of ints
widths=list(map(int,(offsets.split(','))))

# Same logic for below.
input_enc=lines.split("FixedWidthEncoding",1)[1].split(',')[0].split(':')[1].replace(" ", "")
inc_header = lines.split("IncludeHeader",1)[1].split(',')[0].split(':')[1].replace(" ", "")
delimited_enc = lines.split("DelimitedEncoding",1)[1].split(',')[0].split(':')[1].replace(" ", "").strip("}")
#print(delimited_enc)


## Fixed width file generator

In [4]:
import re
tmp2=''
tmp=''
f_width=4
max_len = (f_width*10)+sum(widths)
def fixed_width_generator(input_file):
        # Get the filename
        print("I am here...")
        op_file = input_file.split('/')[-1].split('.')[0]

        # Read the input file
        with open(input_file, encoding='cp1252') as file:
                try:
                        lines = file.read()
                except:
                        print("Codec error : Can't be converted into windows-1252 encoding.")
                        return

        # Declare the output file pointer. Set the encoding as cp1252 stored in f_width_enc(spec file)
        fwp = open(opDir + op_file+'_fixed_width.txt', 'w+', encoding='cp1252')

        # Remove newlines and spaces from lines
        lines = lines.replace("\n", "").replace(" ", "")

        block=''
        file_content = lines
        print(lines)
        # Create the header block
        header_block = ''
        for i in range(len(headers)):
                header_block = header_block + headers[i]+(" "*(widths[i]+(f_width-len(headers[i]))))

        i=0
        cnt=0
        block=''
        is_cp1252=0
        txt_str=file_content
        line_count = 0
        while(txt_str):   
            for k in range(len(widths)):
                print("\ntxt_str:::"+txt_str)
                y = len(txt_str)
                #print("y len gth ::"+str(y))
                #while(i<=y-1):
                cnt=0
                tmp=''
                j=0
                while (cnt<f_width and j<=y-1): 
                    size=0
                    #print("Val is ::")
                    #print(txt_str[j])
                    #print("count is ::")
                    print(cnt)
                    # check for unicode chars
                    myResult1 = re.findall(u"[^\u0000-\u007e]+", txt_str[j])

                    if ( len(myResult1) > 0):
                        #print("Uni")
                        #is_cp1252=1
                        cnt=cnt+2 
                    else:
                        #print("Not")
                        cnt=cnt+1
                        #is_cp1252=0
                    
                    if(cnt!=6 ):
                        tmp=tmp + txt_str[j]
                        #print("tmp is ::")
                        #print(tmp)
                        size=len(tmp)
                        j=j+1
                    elif (cnt == 6 ) :
                        size=len(tmp)
                        #tmp=tmp + " "  # cannot add
                        j=j-1
                
                tmp = tmp + " "*(f_width-size)
                # Update file_content
                txt_str = txt_str[size:]
                #print("Update text str::"+str(txt_str))
                block=block+(tmp)
                #block = block.ljust(5, ' ')
                block=block+(" "*widths[k])
                #print("Block is :::"+block)
                k=k+1

            # Update file_content
            block=block+'\n'
            line_count = line_count+1
            print("line count ==="+str(line_count))
            #txt_str = txt_str[(line_count)*148:]
            print("Updateddddd text str::"+str(txt_str))
#             k=k+1
         
        print("Final block is ::"+block)
        # Write the block to the output file 
        total = header_block + '\n' + block
        fwp.write(total)
        fwp.close()
        

## CSV logic

In [5]:
def generate_csv(input_file):
        # Take out the base filename
        basefile = input_file.split('/')[-1].split('.')[0]
        with open(input_file, encoding=f_width_enc) as f:
                # Create a dataframe having columns as headers
                df = pd.DataFrame(columns=[headers])
                # Read the first line, this will be of headers, skip it
                line = f.readline()
                i = 0
                # Loop to read entire file
                while(line):
                        tmp=[]
                        # Read the next line
                        line = f.readline()
                        # Replace spaced with commas
                        value = " ".join(str(line).split()).replace(" ", ",")
                        # Split to get the values
                        val = value.split(',')
                        # Conditions to handle missing values
                        # If the length is same as the no of columns, put the value straight
                        if ( len(val) == len(headers)):
                                #print("Length is equal to 10 ie "+str(len(val)))
                                df.loc[i] = val
                        elif (len(val)==1 and val[0]==''):   # no further value case
                                continue
                        elif (len(val)<len(headers)):
                                # Case when length is less than headers
                                # Append NA at the remaning elements
                                # Extract till the last value
                                df.loc[i,:len(val)]=val
                                # Put NaN for remaining
                                df.loc[i, len(val):]=np.NaN
                        i=i+1

                # Replace NaN with blanks, this depends upon use case
                df = df.replace(np.nan, '', regex=True)

        #Save to CSV with encoding as utf-8 stored in dem_enc(spec.json file)
        df.to_csv(opDir+basefile+'.csv', encoding=dem_enc, header=True, index=False)

In [6]:
#Iterate through all the files
for filename in os.listdir(opDir):
        # Ignore hidden files
        if not(filename.startswith('.')):
                #print(filename)
                generate_csv(opDir + filename)

In [7]:
# import re
# srr="bwvã000çãHã000çãHã000çãHã000<<<<<çãHã000çãH"
# srr2="bwv„000Á„H„05555500Á„H„000Á„H„000<<<<<Á„H„000Á„H"
# srr3="khvfkbblbflwblf"
# myResult1 = re.findall(u"[^\u0000-\u007e]+", srr)
# myResult1
# myResult2 = re.findall(u"[^\u0000-\u007e]+", srr2)
# myResult2
# #with open('new_file551', 'w+', encoding='cp1252') as file:
# #myResult
    #file.write(srr2)

In [8]:

# numStr = "51111"
# print('Original String :', numStr)
# # Make string right justified of length 4 by padding 3 spaces to left
# numStr = numStr.ljust(5, ' ')
# print('Updated String :', numStr)
# print(len(numStr))


In [9]:
# Iterate each in ipDir
for filename in os.listdir(ipDir):
        if not(filename.startswith('.')):
                print(filename)
                fixed_width_generator(ipDir + filename)
                

test4.txt
I am here...
Ã£000Ã§Ã£HÃ£000Ã§

txt_str:::Ã£000Ã§Ã£HÃ£000Ã§
0
2

txt_str:::000Ã§Ã£HÃ£000Ã§
0
1
2
3

txt_str:::§Ã£HÃ£000Ã§
0
2

txt_str:::£HÃ£000Ã§
0
2
3

txt_str:::£000Ã§
0
2
3

txt_str:::0Ã§
0
1
3

txt_str:::

txt_str:::

txt_str:::

txt_str:::
line count ===1
Updateddddd text str::
Final block is ::Ã£       000Ã            §Ã     £HÃ   £00              0Ã§                                                                    

test5.txt
I am here...
Ã£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HvÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HvÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£H

txt_str:::Ã£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã£HÃ£000Ã§Ã

In [10]:
# zzz="ã000çãHã000"
# s.find(zzz.decode("utf-8"))

# no_special = True
# try:
#     s.encode('latin1')
# except UnicodeEncodeError:
#     no_special = False